In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, ActivityRegularization, Input, Reshape, Flatten, Conv2D
from tensorflow.keras.utils import plot_model, to_categorical

In [18]:
class BubbleEvent:
    def __init__(self, File):
        #temp pixel arrays and event level meta data
        self.FileName = File
        Bot1PixelArray, Bot2PixelArray = BubbleEvent.GetPixelArray(self.FileName) #gets 2d array of pixel intensities
        self.Date, self.Run, self.EventID = BubbleEvent.GetRunInfo(self.FileName) #parses image name to get event info
        self.BubbleCount = 0
        #actual features to use to classify
        self.UsefulEdgeFeature0, self.UsefulEdgeFeature1, self.UsefulEdgeFeature2 = (GetEdgeFeature(
                                        DownSampleTheArray(2, Bot1PixelArray)) + 
                                        GetEdgeFeature(DownSampleTheArray(2, Bot2PixelArray))) #edge detect. sum
        self.UsefulBlobFeature = np.std(GetBlobs(Bot1PixelArray)) + np.std(GetBlobs(Bot2PixelArray)) #blob convalution deviation
        self.CountBlobPeakFeature = GetPeaks(Bot1PixelArray) + GetPeaks(Bot2PixelArray)
    def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray
    def GetRunInfo(File):
        Date = int(File.split("/")[-1].split("_")[0]) #file should be date_run_event
        Run = int(File.split("/")[-1].split("_")[1])
        Event = int("{}{}{}".format(Date, Run,File.split("/")[-1].split("_")[2])) 
        return Date, Run, Event

def OneHotEncode(Y):
    NewY = np.zeros((len(Y), len(set(Y))))
    for Count in set(Y):
        Cut = Y==Count
        NewY[Cut, Count] = 1
    return NewY

In [22]:
EventData = pkl.load(open("Events.p", "rb"))
X=[]
Y=[]
for i in range(0, len(EventData)):
    if(EventData[i].BubbleCount != -1 or EventData[i].CountBlobPeakFeature == 0): #no visible bubbles or good count
        if(EventData[i].BubbleCount>0):
            Y.append(EventData[i].BubbleCount)
        else:
            Y.append(0)
        X.append([EventData[i].UsefulEdgeFeature, EventData[i].UsefulBlobFeature, EventData[i].CountBlobPeakFeature])
X = np.asarray(X)
Y = np.asarray(Y)
NewY = OneHotEncode(Y)

In [35]:
X_Train = X[0:14000, :]
Y_Train = NewY[0:14000, :]
X_Test = X[14000:, :]
Y_Test = NewY[14000:, :]

loss_fn = 'categorical_crossentropy'
optim = 'adam'
batch_size = 500
basic_model = Sequential()
basic_model.add(Input(3))
basic_model.add(Dense(6, activation='relu'))
basic_model.add(Dense(len(set(Y)), activation='softmax'))
basic_model.compile(optimizer = optim,
                    loss = loss_fn,
                    metrics = ['accuracy'])
history = basic_model.fit(X_Train, 
                Y_Train, 
                batch_size = batch_size,
                epochs = 150)
Score = basic_model.evaluate(X_Test, Y_Test, verbose=0)
print(Score)

Epoch 1/150
28/28 [==============================] - 0s 3ms/step - loss: 4.0239 - accuracy: 0.6552
Epoch 2/150
28/28 [==============================] - 0s 3ms/step - loss: 0.9744 - accuracy: 0.4914
Epoch 3/150
28/28 [==============================] - 0s 3ms/step - loss: 0.8434 - accuracy: 0.6519
Epoch 4/150
28/28 [==============================] - 0s 5ms/step - loss: 0.8223 - accuracy: 0.6519
Epoch 5/150
28/28 [==============================] - 0s 3ms/step - loss: 0.8120 - accuracy: 0.6519
Epoch 6/150
28/28 [==============================] - 0s 3ms/step - loss: 0.8016 - accuracy: 0.6537
Epoch 7/150
28/28 [==============================] - 0s 4ms/step - loss: 0.7916 - accuracy: 0.6563
Epoch 8/150
28/28 [==============================] - 0s 4ms/step - loss: 0.7817 - accuracy: 0.6563
Epoch 9/150
28/28 [==============================] - 0s 3ms/step - loss: 0.7714 - accuracy: 0.6563
Epoch 10/150
28/28 [==============================] - 0s 4ms/step - loss: 0.7614 - accuracy: 0.6562
Epoch 11/

28/28 [==============================] - 0s 4ms/step - loss: 0.2229 - accuracy: 0.9581
Epoch 83/150
28/28 [==============================] - 0s 4ms/step - loss: 0.2197 - accuracy: 0.9636
Epoch 84/150
28/28 [==============================] - 0s 3ms/step - loss: 0.2183 - accuracy: 0.9636
Epoch 85/150
28/28 [==============================] - 0s 2ms/step - loss: 0.2165 - accuracy: 0.9636
Epoch 86/150
28/28 [==============================] - 0s 3ms/step - loss: 0.2156 - accuracy: 0.9636
Epoch 87/150
28/28 [==============================] - 0s 6ms/step - loss: 0.2144 - accuracy: 0.9636
Epoch 88/150
28/28 [==============================] - 0s 3ms/step - loss: 0.2132 - accuracy: 0.9636
Epoch 89/150
28/28 [==============================] - 0s 4ms/step - loss: 0.2119 - accuracy: 0.9636
Epoch 90/150
28/28 [==============================] - 0s 3ms/step - loss: 0.2103 - accuracy: 0.9636
Epoch 91/150
28/28 [==============================] - 0s 4ms/step - loss: 0.2091 - accuracy: 0.9636
Epoch 92/150
